In [106]:
#!pip install pyspark


In [37]:

from pyspark.sql.functions import regexp_extract, date_format, to_date, col, countDistinct, count, when


In [28]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.appName("WebServerLogAnalysis").getOrCreate()


In [20]:
# Connect to drive to fetch log files
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
# Load the logs file into a DataFrame
# Define file paths
log_path_jul = "/content/drive/My Drive/access_log_Jul95"
log_path_aug = "/content/drive/My Drive/access_log_Aug95"

# Read both log files
df_jul = spark.read.text(log_path_jul)
df_aug = spark.read.text(log_path_aug)

# Combine both DataFrames
log_df = df_jul.union(df_aug)
log_df.show(5, truncate=False)  # Display first 5 rows


+-----------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                  |
+-----------------------------------------------------------------------------------------------------------------------+
|199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] "GET /history/apollo/ HTTP/1.0" 200 6245                                 |
|unicomp6.unicomp.net - - [01/Jul/1995:00:00:06 -0400] "GET /shuttle/countdown/ HTTP/1.0" 200 3985                      |
|199.120.110.21 - - [01/Jul/1995:00:00:09 -0400] "GET /shuttle/missions/sts-73/mission-sts-73.html HTTP/1.0" 200 4085   |
|burger.letters.com - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/countdown/liftoff.html HTTP/1.0" 304 0               |
|199.120.110.21 - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/missions/sts-73/sts-73-patch-small.gif HTTP/1.0" 200 4179|
+-----------------------

In [30]:
# Parsing the log files correctly according to the Common Log Format
log_pattern = r'(\S+) (\S+) (\S+) \[(.*?)\] "(.*?)" (\d{3}) (\S+)'

parsed_df = log_df.withColumn("remotehost", regexp_extract(col("value"), log_pattern, 1)) \
                  .withColumn("rfc931", regexp_extract(col("value"), log_pattern, 2)) \
                  .withColumn("authuser", regexp_extract(col("value"), log_pattern, 3)) \
                  .withColumn("date", regexp_extract(col("value"), log_pattern, 4)) \
                  .withColumn("request", regexp_extract(col("value"), log_pattern, 5)) \
                  .withColumn("status", regexp_extract(col("value"), log_pattern, 6).cast("int")) \
                  .withColumn("bytes", regexp_extract(col("value"), log_pattern, 7).cast("int")) \
                  .drop("value")  # Remove original column

parsed_df.show(5)

+--------------------+------+--------+--------------------+--------------------+------+-----+
|          remotehost|rfc931|authuser|                date|             request|status|bytes|
+--------------------+------+--------+--------------------+--------------------+------+-----+
|        199.72.81.55|     -|       -|01/Jul/1995:00:00...|GET /history/apol...|   200| 6245|
|unicomp6.unicomp.net|     -|       -|01/Jul/1995:00:00...|GET /shuttle/coun...|   200| 3985|
|      199.120.110.21|     -|       -|01/Jul/1995:00:00...|GET /shuttle/miss...|   200| 4085|
|  burger.letters.com|     -|       -|01/Jul/1995:00:00...|GET /shuttle/coun...|   304|    0|
|      199.120.110.21|     -|       -|01/Jul/1995:00:00...|GET /shuttle/miss...|   200| 4179|
+--------------------+------+--------+--------------------+--------------------+------+-----+
only showing top 5 rows



In [90]:
# Handling Missing and Malformed Data

df_cleaned = parsed_df.withColumn(
    "bytes", when(col("bytes") == "-", "0").otherwise(col("bytes")).cast("int")
)
df_cleaned = df_cleaned.replace("-", None, subset=["rfc931", "authuser"])
df_cleaned = df_cleaned.filter(col("remotehost").isNotNull() & col("date").isNotNull())

df_cleaned.show(5)

+--------------------+------+--------+--------------------+--------------------+------+-----+
|          remotehost|rfc931|authuser|                date|             request|status|bytes|
+--------------------+------+--------+--------------------+--------------------+------+-----+
|        199.72.81.55|  NULL|    NULL|01/Jul/1995:00:00...|GET /history/apol...|   200| 6245|
|unicomp6.unicomp.net|  NULL|    NULL|01/Jul/1995:00:00...|GET /shuttle/coun...|   200| 3985|
|      199.120.110.21|  NULL|    NULL|01/Jul/1995:00:00...|GET /shuttle/miss...|   200| 4085|
|  burger.letters.com|  NULL|    NULL|01/Jul/1995:00:00...|GET /shuttle/coun...|   304|    0|
|      199.120.110.21|  NULL|    NULL|01/Jul/1995:00:00...|GET /shuttle/miss...|   200| 4179|
+--------------------+------+--------+--------------------+--------------------+------+-----+
only showing top 5 rows



In [91]:
# Structuring the data into a format suitable for analysis and presentation

# Extract only the "dd/MMM/yyyy" part from "01/Jul/1995:00:00:01 -0400"
df_cleaned = df_cleaned.withColumn(
    "date", regexp_extract("date", r"(\d{2}/[A-Za-z]{3}/\d{4})", 1)
)

# Convert to proper DateType (YYYY-MM-DD)
df_cleaned = df_cleaned.withColumn("date", to_date("date", "dd/MMM/yyyy"))

# Show results
df_cleaned.select("date").show(5, truncate=False)


+----------+
|date      |
+----------+
|1995-07-01|
|1995-07-01|
|1995-07-01|
|1995-07-01|
|1995-07-01|
+----------+
only showing top 5 rows



In [32]:
# Count the total number of log entries
total_log_entries = df_cleaned.count()

print(f"Total number of log entries: {total_log_entries}")


Total number of log entries: 3461613


In [33]:
# Count of unique hosts
unique_hosts = df_cleaned.select("remotehost").distinct().count()
print(f"Unique hosts: {unique_hosts}")


Unique hosts: 137979


In [97]:
# drop null dates before grouping
df_cleaned = df_cleaned.filter(col("date").isNotNull())

# Date wise unique host counts
datewise_hosts = df_cleaned.groupBy(
    date_format("date", "dd-MMM-yyyy").alias("formatted_date")
).agg(
    countDistinct("remotehost").alias("unique_hosts")
).orderBy("formatted_date")

datewise_hosts.show(truncate=False)

+--------------+------------+
|formatted_date|unique_hosts|
+--------------+------------+
|01-Aug-1995   |2582        |
|01-Jul-1995   |5192        |
|02-Jul-1995   |4859        |
|03-Aug-1995   |3222        |
|03-Jul-1995   |7336        |
|04-Aug-1995   |4191        |
|04-Jul-1995   |5524        |
|05-Aug-1995   |2502        |
|05-Jul-1995   |7383        |
|06-Aug-1995   |2538        |
|06-Jul-1995   |7820        |
|07-Aug-1995   |4108        |
|07-Jul-1995   |6474        |
|08-Aug-1995   |4406        |
|08-Jul-1995   |2898        |
|09-Aug-1995   |4317        |
|09-Jul-1995   |2554        |
|10-Aug-1995   |4523        |
|10-Jul-1995   |4464        |
|11-Aug-1995   |4346        |
+--------------+------------+
only showing top 20 rows



In [100]:
#Average Requests per Host per Day

daily_avg = df_cleaned.groupBy(
    date_format("date", "dd-MMM-yyyy").alias("date")
).agg(
    count("*").alias("total_requests"),
    countDistinct("remotehost").alias("unique_hosts")
).withColumn(
    "avg_requests_per_host",
    (col("total_requests") / col("unique_hosts")).cast("DECIMAL(10,2)")
).orderBy("date")

daily_avg.show(truncate=False)

+-----------+--------------+------------+---------------------+
|date       |total_requests|unique_hosts|avg_requests_per_host|
+-----------+--------------+------------+---------------------+
|01-Aug-1995|33996         |2582        |13.17                |
|01-Jul-1995|64714         |5192        |12.46                |
|02-Jul-1995|60265         |4859        |12.40                |
|03-Aug-1995|41388         |3222        |12.85                |
|03-Jul-1995|89584         |7336        |12.21                |
|04-Aug-1995|59557         |4191        |14.21                |
|04-Jul-1995|70452         |5524        |12.75                |
|05-Aug-1995|31893         |2502        |12.75                |
|05-Jul-1995|94575         |7383        |12.81                |
|06-Aug-1995|32420         |2538        |12.77                |
|06-Jul-1995|100960        |7820        |12.91                |
|07-Aug-1995|57362         |4108        |13.96                |
|07-Jul-1995|87233         |6474        

In [101]:
#Number of 404 Response Codes

df_404 = df_cleaned.filter(col("status") == 404)
count_404 = df_404.count()
print(f"Total number of 404 responses: {count_404}")



Total number of 404 responses: 20901


In [105]:
# Top 15 Endpoints with 404 Responses
top_404_hosts = df_cleaned.filter(df_cleaned.status == 404) \
                         .groupBy("request").count() \
                         .orderBy(col("count").desc()) \
                         .limit(15)

top_404_hosts.show()

+--------------------+-----+
|             request|count|
+--------------------+-----+
|GET /pub/winvn/re...| 2004|
|GET /pub/winvn/re...| 1732|
|GET /shuttle/miss...|  682|
|GET /shuttle/miss...|  426|
|GET /history/apol...|  384|
|GET /history/apol...|  383|
|GET /://spacelink...|  381|
|GET /images/crawl...|  374|
|GET /elv/DELTA/un...|  372|
|GET /history/apol...|  359|
|GET /images/nasa-...|  319|
|GET /shuttle/reso...|  310|
|GET /history/apol...|  304|
|GET /shuttle/reso...|  262|
|GET /shuttle/miss...|  190|
+--------------------+-----+



In [102]:
# Top 15 Hosts with 404 Responses
top_404_hosts = df_cleaned.filter(df_cleaned.status == 404) \
                         .groupBy("remotehost").count() \
                         .orderBy(col("count").desc()) \
                         .limit(15)

top_404_hosts.show()


+--------------------+-----+
|          remotehost|count|
+--------------------+-----+
|hoohoo.ncsa.uiuc.edu|  251|
|piweba3y.prodigy.com|  157|
|jbiagioni.npt.nuw...|  132|
|piweba1y.prodigy.com|  114|
|www-d4.proxy.aol.com|   91|
|piweba4y.prodigy.com|   86|
|scooter.pa-x.dec.com|   69|
|www-d1.proxy.aol.com|   64|
|phaelon.ksc.nasa.gov|   64|
|www-b4.proxy.aol.com|   62|
|dialip-217.den.mm...|   62|
|www-b3.proxy.aol.com|   61|
|www-a2.proxy.aol.com|   60|
|piweba2y.prodigy.com|   59|
|            titan02f|   59|
+--------------------+-----+

